# New York City Taxi Cab Trip - Spark dataframes on HDFS



We look at [the New York City Taxi Cab dataset](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml). This includes every ride made in the city of New York since 2009.

On [this website](http://chriswhong.github.io/nyctaxi/) you can see the data for one random NYC yellow taxi on a single day.

On [this post](http://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/), you can see an analysis of this dataset. Postgres and R scripts are available on [GitHub](https://github.com/toddwschneider/nyc-taxi-data).

## Loading the data

Normally we would read and load this data into memory as a Pandas dataframe.  However in this case that would be unwise because this data is too large to fit in RAM.

The data can stay in the hdfs filesystem but for performance reason we can't use the csv format. The file is large (32Go) and text formatted. Data Access is very slow.

You can convert csv file to parquet with Spark.

```python 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Convert CSV to parquet") \
        .master("spark://b2-120-gra11:7077") \
        .getOrCreate()

df = spark.read.csv("hdfs://b2-120-gra11:54310/data/nyctaxi/2018/yellow*.csv", 
                    header="true",inferSchema="true")

df.write.parquet("hdfs://b2-120-gra11:54310/user/jupyter-navaro_p/2018-yellow.parquet")

spark.stop()
```

## Spark Cluster

A Spark cluster is available and described on this [web interface](http://mas-sdd-rennes2.ovh:8080)

![](images/cluster-overview.png)

The SparkSession is connected to the Spark’s own standalone cluster manager (It is also possible to use YARN). The manager allocate resources across applications. Once connected, Spark acquires executors on nodes in the cluster, which are processes that run computations and store data for your application. Next, it sends your application code (Python file) to the executors. Finally, tasks are sent to the executors to run.

Spark can access to files located on hdfs and it is also possible to access to local files. Example:

## spark-submit.sh

The spark-submit script is used to launch applications on a cluster. It can use all of Spark’s supported cluster managers through a uniform interface so you don’t have to configure your application especially for each one.

Example

In [2]:
%%file wordcount.py

import sys, os
 
from pyspark import SparkContext, SparkConf
 
if __name__ == "__main__":
 
    # create Spark context with Spark configuration
    conf = SparkConf().setAppName("WordCount")
    sc = SparkContext(conf=conf)
 
    # read in text file and split each document into words
    words = sc.textFile("file:///srv/data/sample.txt").flatMap(lambda line: line.strip().lower().replace(".","").split(" "))
 
    # count the occurrence of each word
    wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
 
    wordCounts.saveAsTextFile(os.path.join("output"))

Overwriting wordcount.py


Vous pouvez tester la commande en l'exécutant dans un terminal

```bash
/opt/spark-3.3.1-bin-hadoop3/bin/spark-submit wordcount.py
```

Utilisez `Ctrl-C` pour la stopper en cas de problème

par défaut la sortie se fait sur le système de fichier hdfs 

```
hdfs dfs -cat output/*
```

- **Don't run the python code inside a notebook cell**. Save a python script and launch it from a terminal instead.
In Jupyter notebook you won't see any progress or information if error occurs.
- Documentation of [`spark-submit`](https://spark.apache.org/docs/latest/submitting-applications.html) command shell to run your script on the cluster.
- You can control the log with 
```py
spark.sparkContext.setLogLevel('ERROR')
```
Valid log levels include: ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN

**Try your script with a single file before to do it for whole data.**

**Read carefully your script before, don't submit many times.**

## Some examples that can be run on the cluster

- Here we read the NYC taxi data files of year 2018 and select some variables.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master('spark://b2-120-gra11:7077') \
        .getOrCreate()
spark

In [3]:
columns = ['tpep_pickup_datetime', 'passenger_count', 'payment_type', 'fare_amount', 
           'tip_amount', 'total_amount']
df = (spark.read.parquet("hdfs://localhost:54310/data/nyctaxi/2018.parquet/").select(*columns))

- Sum the total number of passengers

In [ ]:
df.agg({'passenger_count': 'sum'}).collect()

- Average number of passenger per trip`

In [ ]:
df.agg({'passenger_count': 'avg'}).collect()

- How many trip with 0,1,2,3,...,9 passenger`

In [6]:
df.groupby('passenger_count').agg({'*': 'count'}).collect()

[Row(passenger_count=1, count(1)=73072141),
 Row(passenger_count=6, count(1)=2783068),
 Row(passenger_count=3, count(1)=4295075),
 Row(passenger_count=5, count(1)=4602861),
 Row(passenger_count=9, count(1)=275),
 Row(passenger_count=4, count(1)=2029082),
 Row(passenger_count=8, count(1)=313),
 Row(passenger_count=7, count(1)=390),
 Row(passenger_count=2, count(1)=15087976),
 Row(passenger_count=0, count(1)=933067),
 Row(passenger_count=96, count(1)=1),
 Row(passenger_count=192, count(1)=1)]

## Example

How well people tip based on the number of passengers in a cab.  To do this you have to:

1.  Remove rides with zero fare
2.  Add a new column `tip_fraction` that is equal to the ratio of the tip to the fare
3.  Group by the `passenger_count` column and take the mean of the `tip_fraction` column.

- To remove rows
```python
df = df.filter(df.name == 'expression')
```

- To make new columns
```python
df = df.withColumn('var2', df.var0 + df.var1)
```

- To do groupby-aggregations
```python
df.groupBy(df.name).agg({'column-name': 'avg'})
```

When you want to collect the result of your computation, finish with the `.collect()` method.

In [7]:
(df.filter(df.fare_amount > 0)
   .withColumn('tip_fraction', df.tip_amount / df.fare_amount)
   .groupby('passenger_count').agg({'tip_fraction': 'avg'})).collect()


[Row(passenger_count=1, avg(tip_fraction)=0.16703534839637652),
 Row(passenger_count=6, avg(tip_fraction)=0.14819715619095095),
 Row(passenger_count=3, avg(tip_fraction)=0.15911954017995542),
 Row(passenger_count=5, avg(tip_fraction)=0.15291846582994303),
 Row(passenger_count=9, avg(tip_fraction)=0.16675724210066872),
 Row(passenger_count=4, avg(tip_fraction)=0.15219073673118927),
 Row(passenger_count=8, avg(tip_fraction)=0.2569091162309514),
 Row(passenger_count=7, avg(tip_fraction)=0.11830764381318884),
 Row(passenger_count=2, avg(tip_fraction)=0.15288115804135563),
 Row(passenger_count=0, avg(tip_fraction)=0.1687758008957535),
 Row(passenger_count=192, avg(tip_fraction)=0.24333333333333332),
 Row(passenger_count=96, avg(tip_fraction)=0.22666666666666668)]

22/12/06 10:18:04 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/12/06 10:18:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce